In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 4.2 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [3]:
import os
from pyngrok import ngrok

In [4]:
ngrok.kill()

In [5]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://juniper-unvoided-wispily.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://juniper-unvoided-wispily.ngrok-free.dev


True

In [6]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [7]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [8]:
result = stateful_query("簡介比特幣")
print(result)

比特幣（Bitcoin，縮寫：BTC 或 XBT，符號：₿）是一種基於去中心化、採用點對點網路、共識主動性與開放原始碼，並以區塊鏈作為底層技術的加密貨幣。它是一種數位資產，旨在作為交易媒介，無需銀行或中央機構介入。

**起源與創造者**
比特幣的概念最早由一位化名「中本聰」（Satoshi Nakamoto）的匿名人士或團隊於2008年10月31日發表論文中提出。隨後，比特幣的創世區塊於2009年1月3日誕生，標誌著比特幣網路的正式啟動。中本聰於2010年離開比特幣專案後，其真實身份至今仍是個謎。

**主要特點**
比特幣的獨特之處在於其多項關鍵特性：

*   **去中心化**：比特幣網路不受任何單一機構、政府或金融機構的控制，這意味著沒有中央發行機構可以任意增發貨幣或凍結帳戶。
*   **區塊鏈技術**：比特幣所有交易記錄都儲存在一個分散式、公開透明的數位帳本——區塊鏈上。每筆交易都經過網路節點的密碼學驗證，並記錄在公開帳本中。
*   **限量供應**：比特幣的總量上限為2100萬枚，預計在2140年全數挖完。這種稀缺性是其價值主張之一。
*   **點對點交易**：使用者可以透過網路直接交換物品、服務，無需任何銀行、信用卡或線上支付公司等中介機構。
*   **挖礦**：新的比特幣透過「挖礦」過程產生。礦工利用電腦算力解決複雜的數學難題，以驗證並打包交易成為區塊，成功者可獲得新發行的比特幣作為獎勵和交易手續費。
*   **匿名性與透明性**：雖然建立比特幣錢包不需要提供個人資料，但區塊鏈上的所有交易記錄都是公開、透明且可追溯的。這意味著交易地址是匿名的，但交易本身是公開的。

**用途與價值**
比特幣通常被視為一種投資工具、數位黃金的儲值方式，以及在全球範圍內轉移價值的方式。由於其去中心化和限量供應的特性，比特幣在金融危機時期被認為可以避免傳統金融體系的缺陷。然而，它也因其價格波動和監管不確定性而受到批評，在許多國家和學術界，比特幣仍被視為虛擬商品而非法定貨幣。儘管如此，一些國家已嘗試將比特幣列為法定貨幣，例如薩爾瓦多，但也有國家在推行後廢除.


In [9]:
result2 = stateful_query("發明者是誰？")
print(result2)

比特幣的發明者是一位化名為「中本聰」（Satoshi Nakamoto）的匿名人士或團隊。

中本聰於2008年10月31日發表了一篇名為《比特幣：一種點對點電子現金系統》（Bitcoin: A Peer-to-Peer Electronic Cash System）的白皮書，提出了比特幣的概念。 隨後，他在2009年1月3日挖出了比特幣的創世區塊，正式啟動了比特幣網路。

至今為止，中本聰的真實身份仍然是個謎。 雖然有多方猜測和調查試圖揭露其身份，包括將其與一些密碼學家或企業家（如Hal Finney, Dorian Nakamoto, Nick Szabo, Craig Wright等）聯繫起來，但都沒有確鑿的證據能夠證實。 中本聰在2010年後便逐漸淡出比特幣的開發，並將專案交給了其他開發者，此後再未公開露面。 許多人認為，中本聰選擇匿名可能是為了保護個人隱私、安全，並確保比特幣的去中心化本質，使其不受任何個人或組織的控制。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Jan/2026 11:54:46] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"Uc0bc2480f5fd2dd98684ea589fdf9de6","events":[]}
BODY:  {"destination":"Uc0bc2480f5fd2dd98684ea589fdf9de6","events":[{"type":"message","message":{"type":"text","id":"594791899567489570","quoteToken":"eAaatEptFcW6tErKFxgBRMj6zgiIFMfwUd5YTJh0p5ORPheCogVjEph-yKqlBCvz9FT8EK2NLrWrq-a_YlmsetXREr4fJ5AGJ0nmc6B_4Yq33VrpEmrRPUrQzw-dXpnRzbvcIEkd0gq5gSa_V-2sxg","markAsReadToken":"wkqhK24ixuf1nQEXzUR2c2knzOC2EXqvaOgHivlFFtvSRNUOSJkKzNEoQR0rJ6QzK4h5wnjDWr3wtVXkLQtq0MfyHBIGA_B5HejWP8kE5Jc-rgnF8dASN0YyilxFuw7RJeXG318mt_4uwMf1vPUtqUMwasGY-azImRH-SED3J8_NTWoj42V9RJz1cYt6qsh1m-xcCtC5LiS3H1y2_IqIIw","text":"AI 簡介比特幣"},"webhookEventId":"01KDZ8ZB1397NGY10J90PV0GHG","deliveryContext":{"isRedelivery":false},"timestamp":1767354903056,"source":{"type":"user","userId":"Uafbaa1ab617f08637f77d4fdd09c7975"},"replyToken":"12259d116431438382bd06f4a18c1000","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [02/Jan/2026 11:55:07] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"Uc0bc2480f5fd2dd98684ea589fdf9de6","events":[{"type":"message","message":{"type":"text","id":"594791929330794531","quoteToken":"curmWH83gVBUK-5M-sTDGGJ_knlbN7Q3zPZ4X_NuJmHJSwNMo2sCm_VkUSJ5q31dgFKez1OMEgQbWAtb5ilRWIUBhAajGYXAfN-QVFmF8D6mFKWRjXDXl5kmHgV_uJYaUX_zx6spztMmYoTo1KD6UA","markAsReadToken":"PjXlNy5Cuf8QV6hXCpCR5191jKz2lmqrI3MkBfgUCgH12ieceLnLhofWPTHFIdMbJG4zbHfIX2GNu0Cp-pujBTN7TwLhbuT5MnrpBze1YQNdritGVyW8GVfzt7nLnghHd2k-8xRDWMzaVPYuN1mwupr6N2ADeQXI0_oNbYlGl8qGuw-eNL_GWfIQROQfCGTNENXzFSlo4_0Rne9P6n12_g","text":"AI 發明者是誰"},"webhookEventId":"01KDZ8ZVZVJ75VE29GRQ6B2Y59","deliveryContext":{"isRedelivery":false},"timestamp":1767354920635,"source":{"type":"user","userId":"Uafbaa1ab617f08637f77d4fdd09c7975"},"replyToken":"a5c0b2a3d95b4c9a9f692cc832515607","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [02/Jan/2026 11:55:23] "POST / HTTP/1.1" 200 -
